In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import argparse
import sys
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_svmlight_file
import sys
sys.path.append('../')

from cs_opl import op_learning
from experiment_learning import *

In [3]:
data_name = 'satimage'
num_trials = 1
sample_size = 300

if data_name == 'satimage':
    data_name = 'satimage.scale'
elif data_name == 'vehicle':
    data_name = 'vehicle.scale'

alphas = [0.7, 0.4, 0.0]

tau_list = np.zeros(num_trials)
res_ipw3_list = np.zeros((num_trials, len(alphas)))
res_dm_list = np.zeros((num_trials, len(alphas)))
res_dml2_list = np.zeros((num_trials, len(alphas)))

res_ipw3_sn_list = np.zeros((num_trials, len(alphas)))
res_dml2_sn_list = np.zeros((num_trials, len(alphas)))

for trial in range(num_trials):
    X, Y, Y_matrix, train_test_split, classes, N, N_train, N_test = data_generation(data_name, sample_size)

    X_train, X_test = X[train_test_split], X[~train_test_split]

    Y_matrix_train, Y_matrix_test = Y_matrix[train_test_split], Y_matrix[~train_test_split]

    for idx_alpha in  range(len(alphas)):    
        alpha = alphas[idx_alpha]

        pi_behavior, pi_evaluation  = behavior_and_evaluation_policy(X, Y, train_test_split, classes, alpha=alpha)

        pi_behavior_train = pi_behavior[train_test_split]
        pi_evaluation_train, pi_evaluation_test = pi_evaluation[train_test_split], pi_evaluation[~train_test_split]

        tau = true_value(Y_matrix_test, pi_evaluation_test, N_test)
        tau_list[trial] = tau

        perm = np.random.permutation(N_train)
        X_seq_train, Y_matrix_seq_train, pi_behavior_seq_train, pi_evaluation_seq_train = X_train[perm], Y_matrix_train[perm], pi_behavior_train[perm], pi_evaluation_train[perm]

        Y_historical_matrix = np.zeros(shape=(N_train, len(classes)))
        A_historical_matrix = np.zeros(shape=(N_train, len(classes)))

        for i in range(N_train):
            a = np.random.choice(classes, p=pi_behavior_seq_train[i])
            Y_historical_matrix[i, a] = Y_matrix_seq_train[i, a]
            A_historical_matrix[i, a] = 1
            
        estimators = op_learning(X_seq_train, A_historical_matrix, Y_historical_matrix, X_test, classes)
        
        estimators.ipw_est_parameters()
        epol_ipw = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)
        #epol_ipw_sn = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=True)

        estimators.dm_est_parameters()
        epol_dm = estimators.dm_fit(folds=2, algorithm='Ridge')
        
        estimators.dml_est_parameters(folds=2, method='Ridge')
        epol_dml = estimators.dml_fit(folds=2, algorithm='Ridge', self_norm=False)
        epol_dml_sn = estimators.dml_fit(folds=2, algorithm='Ridge', self_norm=True)

        res_ipw3 =  true_value(Y_matrix_test, epol_ipw, N_test)
        res_ipw3_sn =  true_value(Y_matrix_test, epol_ipw_sn, N_test)
        res_dm =  true_value(Y_matrix_test, epol_dm, N_test)
        res_dml2 =  true_value(Y_matrix_test, epol_dml, N_test)
        res_dml2_sn =  true_value(Y_matrix_test, epol_dml_sn, N_test)

        print('trial', trial)
        print('True:', tau)
        print('IPW3:', res_ipw3)
        print('IPW3_SN:', res_ipw3_sn)
        print('DM:', res_dm)
        print('DML2:', res_dml2)
        print('DML2_SN:', res_dml2_sn)

        res_ipw3_list[trial, idx_alpha] = res_ipw3
        res_ipw3_sn_list[trial, idx_alpha] = res_ipw3_sn
        res_dm_list[trial, idx_alpha] = res_dm
        res_dml2_list[trial, idx_alpha] = res_dml2
        res_dml2_sn_list[trial, idx_alpha] = res_dml2_sn

        np.savetxt("exp_results/true_value_%s.csv"%data_name, tau_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_%s.csv"%data_name, res_ipw3_list, delimiter=",")
        np.savetxt("exp_results/res_opl_ipw3_sn_%s.csv"%data_name, res_ipw3_sn_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dm_%s.csv"%data_name, res_dm_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_%s.csv"%data_name, res_dml2_list, delimiter=",")
        np.savetxt("exp_results/res_opl_dml2_sn_%s.csv"%data_name, res_dml２_sn_list, delimiter=",")

KeyboardInterrupt: 

In [9]:
estimators.ipw_est_parameters()
epol_ipw = estimators.ipw_fit(folds=5, algorithm='Ridge', self_norm=False)

/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:224: RuntimeWarning: invalid value encountered in true_divide
  G = G_numer / G_denom
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:236: RuntimeWarning: invalid value encountered in true_divide
  B_x = d_mx / f_x - G * d_fx / f_x
/Users/mkato/opt/anaconda3/lib/python3.7/site-packages/statsmodels/nonparametric/kernel_regression.py:237: RuntimeWarning: invalid value encountered in true_divide
  B_x = (G_numer * d_fx - G_denom * d_mx) / (G_denom**2)


score0 0.01882347957111418
score 0.0019949349178287968
score0 0.01216399138511511
score 0.0016137020804414606
score0 0.005176057145634181
score 0.001122959542806945
score0 0.004822094204914695
score 0.0011034365259414626
score0 0.004789949327580576
score 0.0011017453285379608
score0 0.014890457232702286
score 0.005837516286897005
score0 0.009790080707503548
score 0.00630449385290041
score0 0.0035550227008908633
score 0.007137881889907643
score0 0.0032792174577394185
score 0.007194485154836249
score0 0.003255214635976533
score 0.007199777070036748
score0 0.01759116114625772
score 0.0065833070518284765
score0 0.010936324955617326
score 0.005012039675941806
score0 0.004563296842173791
score 0.00343998538728529
score0 0.004251741318125779
score 0.0033606258186391806
score0 0.004223414456315298
score 0.0033533417437080445
score0 0.020513716702354596
score 0.013401317088555149
score0 0.010172143296950568
score 0.0056312180909365425
score0 0.0046811274204518055
score 0.0028609569768560574
sco

KeyboardInterrupt: 

In [6]:
true_value(Y_matrix_test, epol_dml, N_test)

0.6894002155139826

In [8]:
estimators.res

      fun: -0.6488311060947853
 hess_inv: array([[ 17.31299732,  -4.37389247,  -3.54096183, ...,  -9.21532265,
        -10.38455133,  33.17089309],
       [ -4.37389247,  23.5023711 ,  -9.27002902, ...,   5.38810405,
          7.06867534,   8.37371656],
       [ -3.54096183,  -9.27002902,  20.22345674, ..., -10.03723417,
        -14.33767314,  -6.88259046],
       ...,
       [ -9.21532265,   5.38810405, -10.03723417, ..., 142.23380156,
        151.1348662 , -59.37400152],
       [-10.38455133,   7.06867534, -14.33767314, ..., 151.1348662 ,
        163.89362387, -70.5345726 ],
       [ 33.17089309,   8.37371656,  -6.88259046, ..., -59.37400152,
        -70.5345726 , 467.15308798]])
      jac: array([-1.17719173e-06, -8.94069672e-08,  7.30156898e-07, ...,
        9.98377800e-07,  8.04662704e-07,  2.35438347e-06])
  message: 'Optimization terminated successfully.'
     nfev: 334430
      nit: 245
     njev: 265
   status: 0
  success: True
        x: array([0.16133563, 0.29274354, 0.1921